In [ ]:
%pip install -r "C:\Users\salmaan\projects_github\lending_underwriter\requirements.txt"

  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.1 MB 6.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.1/8.1 MB 5.3 MB/s eta 0:00:02
   --------------- ------------------------ 3.1/8.1 MB 4.9 MB/s eta 0:00:02
   --------------------- ------------------ 4.5/8.1 MB 5.3 MB/s eta 0:00:01
   --------------------------- ------------ 5.5/8.1 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 7.9/8.1 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 5.1 MB/s  0:00:01
Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl (226 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.

In [12]:
%pip install matplotlib

import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Set SHAP to use the Matplotlib backend
#shap.initjs()

# Import our custom functions from the 'src' package
# This works because we ran 'pip install -e .'
from credi_synth_ai.data import generate_synthetic_data
from credi_synth_ai.train import train_model

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'credi_synth_ai'

In [ ]:
# Generate our sample data
df = generate_synthetic_data(num_rows=2000)
df.head()

In [ ]:
# Train the model and get the explainer
# The train function prints the classification report
model_pipeline, explainer, X_test, y_test = train_model(df)

In [ ]:
# Preprocess the X_test data for the explainer
X_test_processed = model_pipeline.named_steps['preprocessor'].transform(X_test)

# Get the feature names from the pipeline
all_feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()

# Create a new DataFrame with the correct feature names
X_test_processed_df = pd.DataFrame(
    X_test_processed, 
    columns=all_feature_names
)

# Calculate SHAP values
shap_values = explainer(X_test_processed_df)

print("SHAP values calculated. Now plotting.")

In [ ]:
# --- Global Feature Importance ---
# This plot answers: "What factors does our model care about most?"
print("SHAP Global Feature Importance (Summary Plot)")
shap.summary_plot(shap_values, X_test_processed_df, plot_type="bar", show=False)
plt.title("Key Drivers of Loan Default Risk")
plt.show()

# This plot shows the direction of the impact
print("\nSHAP Feature Impact (Dot Plot)")
shap.summary_plot(shap_values, X_test_processed_df, show=False)
plt.show()

In [ ]:
# --- Individual Decision Plot ---
# This answers: "Why was this specific MSME (e.g., #10) approved/rejected?"

# Let's find a "defaulted" case
defaulted_index = y_test[y_test == 1].index[0]
defaulted_sample = X_test.loc[defaulted_index]

# Get the processed version of this single sample
sample_processed = model_pipeline.named_steps['preprocessor'].transform(defaulted_sample.to_frame().T)

# Get the SHAP values for just this one sample
sample_shap_values = explainer(sample_processed)

print(f"--- Explaining Decision for Business (Index {defaulted_index}) ---")
print("Original Data:")
print(defaulted_sample)

# Show the waterfall plot
# This shows how each feature "pushed" the score from the base value to the final prediction
shap.waterfall_plot(sample_shap_values[0])